## class decorators
### классы-декораторы
### декораторы классов

In [3]:
from dataclasses import dataclass

@dataclass(slots=True)
class User:
    id: int
    username: str

In [4]:
user = User(id=1, username="john")
print(user)

User(id=1, username='john')


In [5]:
User.__slots__

('id', 'username')

In [6]:
User.mro()

[__main__.User, object]

In [7]:
User.__dataclass_params__

_DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False)

In [8]:
print(repr(user))

User(id=1, username='john')


In [9]:
registry = {}

def register_model(cls):

    registry[cls.__name__] = cls
    cls.__models_registry__ = registry

    return cls

In [10]:
print(registry)

@register_model
class Post:
    pass

print(registry)

{}
{'Post': <class '__main__.Post'>}


In [11]:
print(registry)

@register_model
class Comment:
    pass


print(Comment.__models_registry__)

{'Post': <class '__main__.Post'>}
{'Post': <class '__main__.Post'>, 'Comment': <class '__main__.Comment'>}


In [12]:
Post.__models_registry__

{'Post': __main__.Post, 'Comment': __main__.Comment}

In [13]:
class ModelsRegistry:
    def __init__(self):
        self._registry = {}

    def register(self, decorated_class):
        
        self._registry[decorated_class.__name__] = decorated_class
        decorated_class.__models_registry__ = self._registry

        return decorated_class

In [14]:
models_registry = ModelsRegistry()
print(models_registry._registry)

{}


In [15]:
@models_registry.register
class Client:
    pass

print(models_registry._registry)

@models_registry.register
class Manager:
    pass

print(Manager.__models_registry__)

{'Client': <class '__main__.Client'>}
{'Client': <class '__main__.Client'>, 'Manager': <class '__main__.Manager'>}


In [16]:
class Cached:
    def __init__(self, func):
        self.func = func
        self._cache = {}

    def __call__(self, *args):
        if args not in self._cache:
            self._cache[args] = self.func(*args)
        return self._cache[args]

In [17]:
@Cached
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)

In [18]:
fib

In [19]:
fib._cache

{}

In [20]:
fib(3)

2

In [21]:
fib._cache

{(1,): 1, (0,): 0, (2,): 1, (3,): 2}

In [22]:
fib(10)

55

In [23]:
fib._cache

{(1,): 1,
 (0,): 0,
 (2,): 1,
 (3,): 2,
 (4,): 3,
 (5,): 5,
 (6,): 8,
 (7,): 13,
 (8,): 21,
 (9,): 34,
 (10,): 55}

In [24]:
from functools import wraps


class History:
    def __init__(self, total_items=10):
        self.total_items = total_items
        self.results_history = []
        self.func = None

    def add_history_item(self, result):
        self.results_history.append(result)
        if len(self.results_history) > self.total_items:
            self.results_history.pop(0)

    def wrap(self, func):
        self.func = func
        func.history = self

        @wraps(func)
        def wrapper(*a, **kw):
            result = self.func(*a, **kw)
            self.add_history_item(result)
            return result

        return wrapper
        

    def __call__(self, func):
        return self.wrap(func)
        

In [25]:
@History(total_items=3)
def power(num, e):
    return num ** e

In [26]:
power

<function __main__.power(num, e)>

In [27]:
power(2, 3)

8

In [28]:
power.history

In [29]:
power.history.results_history

[8]

In [30]:
power(4, 2)

16

In [31]:
power(2, 10)

1024

In [32]:
power.history.results_history

[8, 16, 1024]

In [33]:
power.history.total_items

3

In [34]:
power(4, 4)

256

In [35]:
power.history.results_history

[16, 1024, 256]